# Main insights

In [1]:
# - niche products are usually not sold well (i.e. products that have their own ecommerce platform 
# dedicated to them like electrical products and moms-n-kids)
# - unit sales pattern are the same throughout all categories. All categories sells the most in below 100 units
# prices pattern are also the same throughout all categories, except for laptop products (top 3 non-sold rate) 
# with extremely high price range
# - slow-moving categories (i.e. products that are sold little) are usually of electrical products with high price range
# all products in Tiki platform have return and exchange policy in days (7 or 30). In case of slow-moving products, 
# the policy mostly fall into 7 days of return and/or exchange. The nature of electrical products is that they 
# have various latent defects, so the policy only acts as disincentive toward buyers. 
# - the higher number of competitors, the higher number of units sold. this principle applies for categories.
# - rating scores are of the most common pattern, highly-concentrated towards the 2 end of the scoring range.
# - the most commonly used mode of delivery is dropship, that is to utilize 3rd parties to deliver goods. 
# Few categories that are of cross-border are bach-hoa-online, dien-gia-dung, laptop, me-be,... 
# According to 'Tiki university', the sellers of this type of mode usually deal with China, Taiwan, and Korean markets.
# - day_ago_created pattern (i.e. the distribution of the latest update of a product id): 
#     + dien-thoai-may-tinh-bang: most recently posted (1st in non-sold rate, high price range)
#     + me-be: the majority of the product in this category were posted for about 2yrs before, mainly from 1 seller
# - majority of all product ids are backorder. Usually this term means shortage of products, but with 
# further inspection on 'Tiki university', this seems to be a default setting when sellers choose their 
# delivery mode, with 'dropship' default to 'backorder' in inventory type.
# - most sellers have less than 50 products, their revenues are mainly below 300M vnd

In [2]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

In [3]:
df = pd.read_csv('df_prod.csv', engine = 'pyarrow')
df = pd.read_csv('prod_id.csv').merge(df, how = 'right', on = 'id')
df['quantity_sold'] = df.quantity_sold.fillna(0)
df['rev'] = df['quantity_sold']*df['price']
df.head()

,id,category,urlKey,sku,quantity_sold,has_freeship_plus_benefit,partner_rewards_amount,tiki_rewards_amount,total_rewards_amount,store_id,...,fulfillment_type,inventory_status,inventory_type,is_seller_in_chat_whitelist,other_sellers,rating_average,return_and_exchange_policy,review_count,type,rev
0,58533785,8060,thiet-bi-luu-tru,8126201879381,4.0,False,0,2.427907,2.427907,252364,...,dropship,available,backorder,True,"[{'id': 90473, 'name': 'Shop Công Nghệ VITECH2...",0.0,Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,0,simple,1044000.0
1,9370088,8060,thiet-bi-luu-tru,5217569998524,10.0,False,0,0.285302,0.285302,51049,...,dropship,available,backorder,True,"[{'id': 54331, 'name': 'MÁY TÍNH HƯNG THỊNH', ...",5.0,Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,2,simple,306700.0
2,196515362,8060,thiet-bi-luu-tru,4650733278691,3.0,True,16,1.209302,17.527294,98417,...,dropship,available,backorder,True,[],0.0,Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,0,simple,390000.0
3,173230770,8060,thiet-bi-luu-tru,4634285820809,34.0,False,0,0.902326,0.902326,9,...,dropship,available,backorder,True,"[{'id': 60386, 'name': 'VNB STORE', 'link': 'h...",5.0,Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,2,simple,3298000.0
4,249388312,8060,thiet-bi-luu-tru,3044497089293,0.0,False,21,1.581395,22.920307,118699,...,dropship,available,backorder,True,[],0.0,Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,0,configurable,0.0


# 1. Cleaning

In [4]:
# Check non-numeric columns:
# - ['has_buynow', 'inventory_status', 'is_seller_in_chat_whitelist'] -> Drop since these columns only take 1 value
# - 'gift_item_title' -> 2 columns, 'extra_gift' depicts the number of gifts, 'extra_amount' depicts the value of the gifts
# - 'other_sellers' -> 'n_competitors' depicts the number of other sellers
# - ['urlKey', 'has_freeship_plus_benefit', 'fulfillment_type', 'inventory_type', 'return_and_exchange_policy', 'type'] ->
# keep for better description when conducting EDA, but need converting into categorial data (binary columns) 
# when encoding for feature engineering
#   + 'return_and_exchange_policy': rename value to 'dt30', 'dt7', 't30', '7t'

In [5]:
for i in df.select_dtypes(['object', 'bool']).columns:
    print(i)
    print(df[i].value_counts(), '\n')

urlKey
do-dung-va-thiet-bi-nha-tam                    555
sua-chua-nha-cua                               425
vali-phu-kien-vali                             410
mat-kinh                                       400
phu-kien-nam-tui-xach                          352
do-dung-phong-ngu                              346
phu-kien-thoi-trang-nam                        320
trang-suc                                      310
thiet-bi-luu-tru                               310
do-tho-cung                                    306
lam-dep-suc-khoe                               304
thiet-bi-van-phong-thiet-bi-ngoai-vi           301
dong-ho-nu                                     300
laptop-may-vi-tinh                             300
dien-gia-dung                                  298
nhac-cu                                        297
bach-hoa-online                                291
me-be                                          285
thiet-bi-anh-sang                              285
phu-kien-dong-ho        

In [6]:
df = df.drop(columns = ['has_buynow', 'inventory_status', 'is_seller_in_chat_whitelist'])
df['extra_gift'] = df.gift_item_title.str[:1]
df['extra_gift'] = df['extra_gift'].astype(np.int64)
df['extra_amount'] = df.gift_item_title.str[23:-1].str.split('.').str.join('').replace({'': 0})
df['extra_amount'] = df['extra_amount'].astype(np.int64)
df.drop(columns = ['gift_item_title'], inplace = True)

In [7]:
# count the number of 'link' in each row to find the number of competitors
df.other_sellers[1]

"[{'id': 54331, 'name': 'MÁY TÍNH HƯNG THỊNH', 'link': 'https://tiki.vn/cua-hang/may-tinh-hung-thinh', 'logo': '', 'price': 60000, 'product_id': '26053591', 'store_id': 51590}, {'id': 26446, 'name': 'Tin Học Hoàng Kim', 'link': 'https://tiki.vn/cua-hang/tin-hoc-hoang-kim', 'logo': 'bf/ce/4e/e995a297d24c3207a2856b64bee2ce22.jpg', 'price': 80000, 'product_id': '30701494', 'store_id': 25820}, {'id': 20367, 'name': 'VI TÍNH NAM TRUNG HẢI', 'link': 'https://tiki.vn/cua-hang/vi-tinh-nam-trung-hai', 'logo': 'e0/4a/74/e7144f9e9f87ee90acb39c1068e9b422.png', 'price': 35770, 'product_id': '107882248', 'store_id': 19857}, {'id': 76796, 'name': 'Vietstore24h', 'link': 'https://tiki.vn/cua-hang/vietstore24h', 'logo': '5e/db/e6/78033ffbeda4d151d5395a50731dce9f.png', 'price': 49400, 'product_id': '143340499', 'store_id': 71636}, {'id': 265319, 'name': 'Shop Phân Phối Camera', 'link': 'https://tiki.vn/cua-hang/shop-phan-phoi-camera', 'logo': '37/46/d5/442b778beafb23a44b6509f9fbffd5a8.jpg', 'price': 450

In [8]:
df['n_competitors'] = df.other_sellers.str.count('link')
df.drop(columns = ['other_sellers'], inplace = True)

In [9]:
# drop 'total_rewards_amount' since this is the sum of 'partner_rewards_amount' and 'tiki_rewards_amount'
df.drop(columns = ['total_rewards_amount'], inplace = True)

In [10]:
df.return_and_exchange_policy.value_counts()

Đổi trả trong<br><b>30 ngày</b><br>nếu sp lỗi.     5495
 Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.     3651
Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.      2597
Trả hàng trong<br><b>30 ngày</b><br>nếu sp lỗi.    1304
Trả hàng trong<br><b>7 ngày</b><br>nếu sp lỗi.      534
Name: return_and_exchange_policy, dtype: int64

In [11]:
df['return_and_exchange_policy'] = \
df['return_and_exchange_policy'].replace(
    {'Đổi trả trong<br><b>30 ngày</b><br>nếu sp lỗi.': 'dt30',
     ' Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.': 'dt7',
     'Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.': 'dt7',
     'Trả hàng trong<br><b>30 ngày</b><br>nếu sp lỗi.': 't30',
     'Trả hàng trong<br><b>7 ngày</b><br>nếu sp lỗi.': 't7'}
)
df.return_and_exchange_policy.value_counts()

dt7     6248
dt30    5495
t30     1304
t7       534
Name: return_and_exchange_policy, dtype: int64

In [12]:
# df.to_csv('eda_cleaned.csv', index = False)

# 2. EDA

In [13]:
df[
    df.select_dtypes(['int64', 'float64']).columns
].describe().round()

,id,category,sku,quantity_sold,partner_rewards_amount,tiki_rewards_amount,store_id,day_ago_created,original_price,price,discount,discount_rate,rating_average,review_count,rev,extra_gift,extra_amount,n_competitors
count,13581.0,13581.0,1.358100e+04,13581.0,13581.0,13581.0,13581.0,13581.0,13581.0,13581.0,13581.0,13581.0,13581.0,13581.0,1.358100e+04,13581.0,13581.0,13581.0
mean,128128607.0,11407.0,5.481043e+12,65.0,10.0,11.0,131194.0,1674.0,1426252.0,1324813.0,101441.0,8.0,3.0,12.0,1.796390e+07,0.0,1794.0,1.0
std,70634644.0,10016.0,2.598130e+12,332.0,33.0,33.0,99156.0,26829.0,4965763.0,4760732.0,563017.0,15.0,2.0,58.0,1.404502e+08,0.0,52649.0,4.0
min,136558.0,958.0,1.000451e+12,0.0,0.0,0.0,9.0,0.0,1000.0,1000.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0
25%,72049059.0,2015.0,3.221393e+12,0.0,0.0,1.0,35111.0,350.0,99000.0,89700.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0
50%,131174913.0,8337.0,5.480917e+12,3.0,0.0,2.0,120859.0,606.0,250000.0,225000.0,0.0,0.0,4.0,1.0,6.906900e+05,0.0,0.0,0.0
75%,188576770.0,21054.0,7.720422e+12,24.0,0.0,6.0,219133.0,954.0,699000.0,629000.0,18346.0,10.0,5.0,5.0,5.166000e+06,0.0,0.0,0.0
max,263083810.0,49650.0,9.999961e+12,13186.0,874.0,235.0,326330.0,737994.0,100000000.0,100000000.0,24146000.0,99.0,5.0,2816.0,6.849572e+09,4.0,4949000.0,110.0


## 2.1. Non-sold rate

$\text{non-sold rate} = \frac{\text{number of product id that have 0 quantity sold}}{\text{total number of product id}}$

In [14]:
selling_rate = \
df.groupby('urlKey')[['id']] \
.agg('count').rename({'id': 'total'}, axis = 1) \
.merge(
    df[df['quantity_sold'] == 0].groupby('urlKey')[['id']] \
        .agg('count').rename({'id': 'total_non_sold'}, axis = 1),
    how = 'inner',
    left_index = True,
    right_index = True)

selling_rate['non_sold_rate'] = selling_rate.total_non_sold / selling_rate.total
selling_rate = selling_rate.sort_values(by = ['non_sold_rate'], ascending = False)
selling_rate.head(10)

,total,total_non_sold,non_sold_rate
urlKey,,,
dien-thoai-may-tinh-bang,230,226,0.982609
me-be,285,256,0.898246
laptop,229,197,0.860262
may-bay-camera-va-phu-kien,41,35,0.853659
bach-hoa-online,291,248,0.852234
ong-kinh-lens,89,74,0.831461
laptop-may-vi-tinh,300,242,0.806667
pc-may-tinh-bo,80,60,0.750000
dien-gia-dung,298,219,0.734899


In [15]:
selling_rate.tail(10)

,total,total_non_sold,non_sold_rate
urlKey,,,
sua-chua-nha-cua,425,35,0.082353
ngoai-troi-san-vuon,272,20,0.073529
thiet-bi-van-phong-thiet-bi-ngoai-vi,301,21,0.069767
noi-that,222,15,0.067568
trang-tri-nha-cua,203,13,0.064039
phu-kien-thoi-trang-nu,239,15,0.062762
phu-kien-the-thao,214,13,0.060748
trang-suc,310,16,0.051613
phong-an,197,10,0.050761


## 2.2. Category and sales/price/n_competitors

In [16]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [17]:
# All category has the same sales pattern, no one stands out.

In [18]:
category_sales_trace = []
outliers = df.quantity_sold.quantile(0.99)
for i in df.urlKey.unique():
    category_sales_trace.append(
        go.Histogram(x = df[(df.urlKey == i) & (df.quantity_sold < outliers)].quantity_sold,
                     opacity = 0.7,
                     name = '<i>{}<i>'.format(i))
    )

go.Figure(
    data = category_sales_trace,
    layout = go.Layout(title = '<b>Histogram of Quantity Sold by Category<b>',
                       barmode = 'overlay',
                       xaxis = dict(title = '<b><i>sales(unit)<i><b>'),
                       yaxis = dict(title = '<b><i>frequency<i><b>'))
).show()

In [19]:
# Top 10 categories: can be considered fast-moving

In [20]:
category_sales = \
df.groupby(['category', 'urlKey'])[['quantity_sold']] \
.agg([np.mean, np.std, np.min, percentile(25), np.median, percentile(75), np.max]) \
.astype(np.int64).sort_values(by = [('quantity_sold', 'mean')], ascending = False).reset_index().set_index('category')
category_sales.style.background_gradient(subset = [('quantity_sold', 'mean')], vmax = 200)

In [21]:
category_price_trace = []
outliers = df.price.quantile(0.99)
for i in df.urlKey.unique():
    category_price_trace.append(
        go.Histogram(x = df[(df.urlKey == i) & (df.price < outliers)].price,
                     opacity = 0.7,
                     name = '<i>{}<i>'.format(i))
    )

go.Figure(
    data = category_price_trace,
    layout = go.Layout(title = '<b>Price pattern across all categories<b>',
                       barmode = 'overlay',
                       xaxis = dict(title = '<b><i>price(vnd)<i><b>'),
                       yaxis = dict(title = '<b><i>frequency<i><b>'))
).show()

In [22]:
# - Low-sales categories usually have the mean price range very high.
# - Top products usually is of tech-products, people usually opt for other platforms that are major in this category

In [23]:
category_price = \
df.groupby(['category', 'urlKey'])[['price']] \
.agg([np.mean, np.std, np.min, percentile(25), np.median, percentile(75), np.max]) \
.astype(np.int64).sort_values(by = [('price', 'mean')], ascending = False).reset_index().set_index('category')
category_price.style.background_gradient(subset = [('price', 'mean')], vmax = 2000000)

In [24]:
# Check the prices of slow-moving categories
category_price.loc[
    category_sales[category_sales[('quantity_sold', 'mean')] < 10].index
].sort_values(by = [('price', 'mean')], ascending = False)

urlKey     price            \
                                                           mean       std   
category                                                                    
28806                                         may-anh  28931078  28026429   
8095                                           laptop  20667533   9823828   
2757                                    ong-kinh-lens  17661977  13355703   
8093                                   pc-may-tinh-bo  16348440  21831290   
28794                               ong-kinh-ong-ngam   2912416   4290008   
28814                               thiet-bi-anh-sang   1746190   2331560   
1840                             balo-tui-dung-bao-da   1669630   2325703   
28834     camera-hanh-trinh-action-camera-va-phu-kien   1635267   1985577   
5337                             tui-xach-cong-so-nam   1601242   1809848   
28822                              thiet-bi-quay-phim   1003796   1662242   
20908                                lam-dep-suc-khoe    981516   2388709   
8337                                   giay-boots-nam    811932   1002215   
24258                                 dung-cu-leo-nui    782937   1316719   
21166                                           me-be    576168    244136   
49650                         tui-bao-tu-tui-deo-bung    446463    964048   
8047                       may-bay-camera-va-phu-kien    437296    868978   
20824                                   dien-gia-dung    367736    235465   
21268                                nha-cua-doi-song    355840    239968   
21074                              laptop-may-vi-tinh    328295    191363   
20766                                 bach-hoa-online    281808    395020   
21054                        dien-thoai-may-tinh-bang    115983     96332   

                                                                   
            amin percentile_25    median percentile_75       amax  
category                                                           
28806       8900        592500  21995000      49997500   89349000  
8095       75000      15690000  19490000      23297000   84990000  
2757      175000       9490000  14259000      20990000   78699000  
8093      129000       5409900   8875000      14117750  100000000  
28794      44000        490000   1370000       4114000   31125000  
28814       5000        270000    840000       2299999   17105000  
1840        2470        181250    800555       2437499   16749999  
28834       6000        241750    549500       2804250    7538000  
5337       59000        537900    928500       2492500   17999000  
28822       2500         83749    242500        999750    8700000  
20908      19000        161000    453000        860500   19100000  
8337       44000        299000    589500        880000    7000000  
24258       3900         35000    204500        850500    5040000  
21166      51000        460299    583947        722085    1959531  
49650      25000         71275    197000        396000    7499000  
8047       20000         42000     78000        226000    3170000  
20824      60000        175500    301500        548250    1004076  
21268      45000        158250    299450        493350     999999  
21074      50000        197750    298798        416853    1219243  
20766      42000        125000    185400        341000    5639000  
21054      41000         63000     89000        120750     619000

In [25]:
df[df.category.isin(category_sales[category_sales[('quantity_sold', 'mean')] < 10].index)] \
[['return_and_exchange_policy']].value_counts()

return_and_exchange_policy
dt7                           1595
t30                           1300
t7                             532
dt30                           104
dtype: int64

In [26]:
df.groupby('urlKey')[['n_competitors', 'quantity_sold']].agg('sum').reset_index().head()

,urlKey,n_competitors,quantity_sold
0,bach-hoa-online,2,124.0
1,balo,68,18667.0
2,balo-cap-tui-chong-soc-laptop,95,13440.0
3,balo-tui-dung-bao-da,13,656.0
4,camera-giam-sat,378,35576.0


In [27]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Bar(name = 'Number of Competitors', 
           x = df.groupby('urlKey')[['n_competitors', 'quantity_sold']].agg('sum').reset_index()["urlKey"], 
           y = df.groupby('urlKey')[['n_competitors', 'quantity_sold']].agg('sum').reset_index()["n_competitors"], 
           yaxis = 'y')
)

fig.add_trace(
    go.Scatter(
        x = df.groupby('urlKey')[['n_competitors', 'quantity_sold']].agg('sum').reset_index()["urlKey"],
        y = df.groupby('urlKey')[['n_competitors', 'quantity_sold']].agg('sum').reset_index()['quantity_sold'],
        name = 'Quantity Sold'
    ),
    secondary_y = True
)

# Add figure title
fig.update_layout(
    title_text = "<b>Number of Competitors vs Quantity Sold by Category<b>"
)

# Set x-axis title
fig.update_xaxes(title_text = "<b>category<b>")

# Set y-axes titles
fig.update_yaxes(title_text = "<b># competitors</b>", secondary_y = False, color = 'blue')
fig.update_yaxes(title_text = "<b>unit</b>", secondary_y = True, color = 'red')

fig.show()

## Correlations

In [28]:
# - High correlation between price/og price and tiki_rewards_amount, quite understandable since the more 
# you spend on buying something, the more redeem points Tiki gives you back. This acts as incentives for 
# better come-back flow from the buyers.
# - While that is the case for Tiki, 3rd parties partners give less incentives, the correlation between that 
# and price/og price just at the low-mid range ~0.2.
# - Medium correlation between discount and price/og price, the higher the og price, the higher the discount. 
# - High correlation between quantity_sold and review_count, this proves that the buyers on the platform are 
# quite active in giving reviews/feedbacks on the bought products. But this doesn't necessarily reflect everything 
# about the relationship between these two. It is advisable to filter out all the 0-valued quantity_sold, then 
# read the correlation between the two and rating average to see how the buyers interact.

In [29]:
df[
    df.select_dtypes(['int64', 'float64']).columns
].corr().style.background_gradient()

,id,category,sku,quantity_sold,partner_rewards_amount,tiki_rewards_amount,store_id,day_ago_created,original_price,price,discount,discount_rate,rating_average,review_count,rev,extra_gift,extra_amount,n_competitors
id,1.000000,0.066984,-0.010388,-0.134990,0.095517,0.114797,0.422716,-0.082141,0.111901,0.106085,0.089924,0.065062,-0.173493,-0.144265,-0.095389,-0.007280,0.011978,-0.163512
category,0.066984,1.000000,0.010363,-0.098464,-0.086041,-0.036692,0.087521,-0.023012,-0.026192,-0.020257,-0.059726,-0.107946,-0.227937,-0.097760,-0.059377,-0.044680,-0.008365,-0.132159
sku,-0.010388,0.010363,1.000000,0.005000,-0.003973,0.004173,0.002835,-0.020267,0.003085,0.001596,0.013711,-0.016341,0.004072,0.001909,0.005480,-0.008463,0.001083,-0.000150
quantity_sold,-0.134990,-0.098464,0.005000,1.000000,-0.023350,-0.052452,-0.104761,-0.002808,-0.041625,-0.043408,-0.000084,0.151489,0.170623,0.870622,0.665317,0.004930,-0.001702,0.049067
partner_rewards_amount,0.095517,-0.086041,-0.003973,-0.023350,1.000000,0.265293,-0.035400,-0.003468,0.222093,0.203351,0.239356,0.114335,-0.052435,-0.013645,0.035500,0.037833,0.016184,-0.028055
tiki_rewards_amount,0.114797,-0.036692,0.004173,-0.052452,0.265293,1.000000,-0.090890,0.039199,0.910372,0.908499,0.347363,-0.051678,-0.218474,-0.052038,0.016587,-0.001461,0.083086,-0.055325
store_id,0.422716,0.087521,0.002835,-0.104761,-0.035400,-0.090890,1.000000,-0.044023,-0.065060,-0.065041,-0.023849,-0.059134,-0.154425,-0.112866,-0.082632,-0.062678,-0.028155,0.014387
day_ago_created,-0.082141,-0.023012,-0.020267,-0.002808,-0.003468,0.039199,-0.044023,1.000000,0.026647,0.027365,0.003628,-0.010236,-0.001177,-0.004528,0.001950,0.018657,0.000479,-0.001382
original_price,0.111901,-0.026192,0.003085,-0.041625,0.222093,0.910372,-0.065060,0.026647,1.000000,0.994185,0.413336,-0.012316,-0.183186,-0.040884,0.019952,0.001800,0.086732,-0.047028
price,0.106085,-0.020257,0.001596,-0.043408,0.203351,0.908499,-0.065041,0.027365,0.994185,1.000000,0.312874,-0.047334,-0.185002,-0.043503,0.012655,-0.001659,0.083769,-0.046024


## 2.3. Rating scores, Reviews, and Quantity sold

In [30]:
# Rating_average score does not go well with the other 2, this also happens quite a lot in other fields, 
# the distribution tends to be bias to the 2 end of the scoring range. Since people usually score with 
# their hunch, not through some systematical rubrik.

In [31]:
df[df['quantity_sold'] != 0] \
[['quantity_sold', 'rating_average', 'review_count']] \
.corr().style.background_gradient()

,quantity_sold,rating_average,review_count
quantity_sold,1.000000,0.110652,0.868217
rating_average,0.110652,1.000000,0.124471
review_count,0.868217,0.124471,1.000000


In [32]:
go.Figure(data = go.Histogram(x = df[df['quantity_sold'] != 0]['rating_average']),
          layout = go.Layout(title = '<b>Rating Scores of products that have been bought at least once<b>',
                             xaxis = dict(title = '<b><i>scores<i><b>'),
                             yaxis = dict(title = '<b><i>frequency<i><b>'))
         ).show()

## 2.4. Category vs Fulfillment

In [33]:
df.pivot_table(index = 'urlKey', columns = 'fulfillment_type', values = 'sku', aggfunc = 'count', fill_value = 0) \
.reset_index().head()

fulfillment_type,urlKey,cross_border,dropship,seller_delivery,tiki_delivery
0,bach-hoa-online,291,0,0,0
1,balo,0,166,2,34
2,balo-cap-tui-chong-soc-laptop,0,257,0,19
3,balo-tui-dung-bao-da,0,190,0,12
4,camera-giam-sat,0,180,0,22


In [34]:
px.bar(df.pivot_table(index = 'urlKey', columns = 'fulfillment_type', values = 'sku', aggfunc = 'count', fill_value = 0).reset_index(), 
       x = "urlKey", 
       y = ["cross_border", "dropship", "seller_delivery", 'tiki_delivery'], 
       title = "<b>Category by mode of delivery<b>").show()

## 2.5. Category vs days ago created

In [35]:
day_trace = []
outliers = df.day_ago_created.quantile(0.99)
for i in df.urlKey.unique():
    day_trace.append(
        go.Histogram(x = df[(df.urlKey == i) & (df.day_ago_created < outliers)].day_ago_created,
                     opacity = 0.7,
                     name = '<i>{}<i>'.format(i))
    )

go.Figure(
    data = day_trace,
    layout = go.Layout(title = '<b>Day_ago_created pattern across all categories<b>',
                       barmode = 'overlay',
                       xaxis = dict(title = '<b><i>days<i><b>'),
                       yaxis = dict(title = '<b><i>frequency<i><b>'))
).show()

In [36]:
df[
    (df.urlKey == 'me-be') & ((df.day_ago_created >= 600) & (df.day_ago_created <= 649))
].store_id.value_counts()

206482    167
206481      6
Name: store_id, dtype: int64

In [37]:
df.groupby(['category', 'urlKey'])[['day_ago_created']] \
.agg([np.mean, np.std, np.min, percentile(25), np.median, percentile(75), np.max]) \
.astype(np.int64).sort_values(by = [('day_ago_created', 'mean')], ascending = False).reset_index().set_index('category') \
.style.background_gradient(subset = [('day_ago_created', 'mean')], vmax = 5000)

## 2.6. Category and Inventory status

In [38]:
df.inventory_type.value_counts()

backorder    12530
instock       1051
Name: inventory_type, dtype: int64

In [39]:
px.bar(df.pivot_table(index = 'urlKey', columns = 'inventory_type', values = 'sku', aggfunc = 'count', fill_value = 0).reset_index(), 
       x = "urlKey", 
       y = ["backorder", "instock"], 
       title = "<b>Inventory Status of the Number of products by category<b>").show()

## 2.7. Category and Product Type

In [40]:
# Configurable indicates that the product has variants, while simple indicates that it is unique.

In [41]:
df.type.value_counts()

simple          6890
configurable    6691
Name: type, dtype: int64

In [42]:
px.bar(df.pivot_table(index = 'urlKey', columns = 'type', values = 'sku', aggfunc = 'count', fill_value = 0).reset_index(), 
       x = "urlKey", 
       y = ["configurable", "simple"], 
       title = "<b>Product types of each Category<b>").show()

## 2.8. Category and Extra gift

In [43]:
extra = \
df.groupby('urlKey')[['extra_gift', 'extra_amount']].agg('sum').reset_index()
extra.head()

,urlKey,extra_gift,extra_amount
0,bach-hoa-online,0,0
1,balo,16,688000
2,balo-cap-tui-chong-soc-laptop,4,356000
3,balo-tui-dung-bao-da,0,0
4,camera-giam-sat,5,64000


In [44]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Bar(name = 'Extra Amount', 
           x = extra["urlKey"], 
           y = extra["extra_amount"], 
           yaxis = 'y')
)

fig.add_trace(
    go.Scatter(
        x = extra["urlKey"],
        y = extra['extra_gift'],
        name = 'Number of extra gifts'
    ),
    secondary_y = True
)

# Add figure title
fig.update_layout(
    title_text = "<b>Extra gifts by category<b>"
)

# Set x-axis title
fig.update_xaxes(title_text = "<b>category<b>")

# Set y-axes titles
fig.update_yaxes(title_text = "<b>Amount(vnd)</b>", secondary_y = False, color = 'blue')
fig.update_yaxes(title_text = "<b>Unit extra</b>", secondary_y = True, color = 'red')

fig.show()

## 2.9. Sellers (Store_id)

In [45]:
# Total number of sellers
len(df.store_id.unique())

3555

In [46]:
# The amount of revenue each store generates, the number of products that each store put on their page, 
# and the average recency of all products of each store.

In [47]:
store_data = \
df.groupby(['store_id'])[['rev', 'sku', 'day_ago_created']] \
.agg({'rev': 'sum', 'sku': 'count', 'day_ago_created': 'mean'}).round().astype(np.int64) \
.rename({'rev': 'revenue(vnd)', 'sku': 'n_product(unit)', 'day_ago_created': 'mean_recency(day)'}, axis = 1)
store_data.head()

,revenue(vnd),n_product(unit),mean_recency(day)
store_id,,,
9,968763000,14,1810
48,64460000,3,601
71,448008500,60,1011
72,10899000,1,1441
85,13525000,3,2150


In [48]:
# Probably have to cluster these sellers into groups, either by splitting through quantile, 
# cumulative % amount of revenue generates, or we can just apply kmeans.

In [49]:
px.scatter_3d(
    store_data[(store_data['mean_recency(day)'] < store_data['mean_recency(day)'].quantile(0.99)) & 
               (store_data['revenue(vnd)'] < store_data['revenue(vnd)'].quantile(0.99))],
    x = 'revenue(vnd)',
    y = 'n_product(unit)',
    z = 'mean_recency(day)'
)

### Try using Kmeans

In [50]:
from sklearn.cluster import KMeans
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("error")

In [51]:
param_grid = \
{
    'n_clusters': [x for x in range(2, 10)],
    'algorithm': ['lloyd', 'elkan', 'auto', 'full']
}
param_list = list(ParameterGrid(param_grid))
len(param_list)

32

In [52]:
finetune_inertia = pd.DataFrame(param_list)
finetune_inertia.head()

,algorithm,n_clusters
0,lloyd,2
1,lloyd,3
2,lloyd,4
3,lloyd,5
4,lloyd,6


In [53]:
X = store_data[
                (store_data['mean_recency(day)'] < store_data['mean_recency(day)'].quantile(0.99)) & 
                (store_data['revenue(vnd)'] < store_data['revenue(vnd)'].quantile(0.99))
              ].values
X_norm = StandardScaler().fit_transform(X)

inertia = []
for params in param_list:
    try:
        kmeans = KMeans(n_clusters = params['n_clusters'], random_state = 46, 
                        tol = 1e-5, algorithm = params['algorithm']).fit(X_norm)
        inertia.append(kmeans.inertia_)
    except:
        inertia.append(np.nan)

In [54]:
# Only 1 algorithm returns result, elkan. The elbow break is at 4-5 clusters. 
# However, the proportion of labels among all the data points is heavily disproportionate.

In [55]:
finetune_inertia['inertia'] = inertia
finetune_inertia = finetune_inertia[~(finetune_inertia.inertia.isna())].sort_values(by = ['inertia'], ascending = False)
finetune_inertia

,algorithm,n_clusters,inertia
8,elkan,2,7568.461485
9,elkan,3,5231.214635
10,elkan,4,3441.503936
11,elkan,5,2850.327935
12,elkan,6,2298.587323
13,elkan,7,1920.197648
14,elkan,8,1679.372195
15,elkan,9,1487.092227


In [56]:
test = \
KMeans(n_clusters = 4, random_state = 46, tol = 1e-5, 
       algorithm = 'elkan').fit(store_data.values)
store_data['group'] = test.labels_
print(test.inertia_)

7.075285279923261e+19


In [57]:
store_data.group.value_counts()

0    3510
3      40
2       4
1       1
Name: group, dtype: int64

### Split by eye

In [58]:
store_data.drop(['group'], axis = 1, inplace = True)
store_data['group_rev'] = np.where(store_data['revenue(vnd)'] <= 500000000, 'L', 'H') # Low, High
store_data['group_nprod'] = np.where(store_data['n_product(unit)'] <= 50, 'F', 'M') # Few, Many
store_data['group_rec'] = np.where(store_data['mean_recency(day)'] <= 1000, 'N', 'O') # New, Old
store_data['group'] = store_data['group_rev'] + store_data['group_nprod'] + store_data['group_rec'] # In order rev, nprod, recency
store_data.head()

,revenue(vnd),n_product(unit),mean_recency(day),group_rev,group_nprod,group_rec,group
store_id,,,,,,,
9,968763000,14,1810,H,F,O,HFO
48,64460000,3,601,L,F,N,LFN
71,448008500,60,1011,L,M,O,LMO
72,10899000,1,1441,L,F,O,LFO
85,13525000,3,2150,L,F,O,LFO


In [59]:
px.scatter_3d(
    store_data[(store_data['mean_recency(day)'] < store_data['mean_recency(day)'].quantile(0.99)) & 
               (store_data['revenue(vnd)'] < store_data['revenue(vnd)'].quantile(0.99))],
    x = 'revenue(vnd)',
    y = 'n_product(unit)',
    z = 'mean_recency(day)',
    color = 'group'
)

In [60]:
# store_data.reset_index().to_csv('store_data.csv', index = False)